# Ocean Heat Content: videos

$OHC = \Delta T \times c_p \times \rho \times V$

unit conversion to SI
- $c_p$: erg/g/K = 1e-7J / 1e-3kg / K = 1e-4 J/kg/K $\rightarrow$ 3996 J/kg/K
- $\rho$: g/cm^3 = 1e3 kg/m^3

In [ ]:
import os
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import OHC_integrals, trend_global_levels
from maps import map_robinson, map_eq_earth
from grid import create_dz_mean
from paths import CESM_filename, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rcp, path_results
from regions import boolean_mask, regions_dict
from plotting import discrete_cmap, shifted_color_map
from constants import cp_sw, km, spy
from timeseries import IterateOutputCESM
from xr_integrate import  xr_vol_int
from xr_regression import xr_linear_trend, xr_linear_trends_2D, ocn_field_regression
from xr_DataArrays import create_xr_DataArray, xr_DZ, xr_AREA, xr_HTN, xr_LATS

In [ ]:
# MASK = xr.open_dataset(file_ex_ocn_ctrl, decode_times=False).REGION_MASK

# fixing OHC.py output

In [ ]:
# xr.set_options(file_cache_maxsize=16)

In [ ]:
# # copying OHc files, removing unnecessary coords that are not perfectly aligned across years
# for run in ['ctrl', 'rcp']:
#     for mask_nr in range(13):
#         for y in range(2100):
#             file_out = f'{path_samoc}/OHC/OHC_integrals_{regions_dict[mask_nr]}_{run}_{y}.nc'
#             file_new = f'{path_samoc}/OHC/OHC_integrals_{regions_dict[mask_nr]}_{run}_new_{y}.nc'
#             if os.path.exists(file_out)==True and os.path.exists(file_new)==False:
#                 print(run, mask_nr, y)
#                 ds = xr.open_dataset(file_out, decode_times=False)
#                 if 'ULONG' in ds.coords and 'ULAT' in ds.coords:
#                     ds = ds.drop(['ULONG', 'ULAT'])
#                 ds.to_netcdf(file_new)

In [ ]:
# %%time
# # 1:30 minutes for ctrl, 35 sec for rcp
# for run in ['ctrl', 'rcp']:
#     for mask_nr in np.arange(1,13):
#         print(run, mask_nr)
#         combined = xr.open_mfdataset(f'{path_samoc}/OHC/OHC_integrals_{regions_dict[mask_nr]}_{run}_new_*.nc',
#                                      concat_dim='time', coords='minimal')
#         combined.to_netcdf(f'{path_samoc}/OHC/OHC_integrals_{regions_dict[mask_nr]}_{run}.nc')

# Loading data

In [ ]:
ctrl   = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_ctrl.nc'  , decode_times=False)
ctrl_A = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Atlantic_Ocean_ctrl.nc', decode_times=False)
ctrl_P = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Pacific_Ocean_ctrl.nc' , decode_times=False)
ctrl_I = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Indian_Ocean_ctrl.nc'  , decode_times=False)
ctrl_M = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Mediterranean_ctrl.nc' , decode_times=False)
ctrl_S = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Southern_Ocean_ctrl.nc', decode_times=False)
ctrl_N = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Arctic_Ocean_ctrl.nc'  , decode_times=False)

# something is still wrong in CTRL year 205
for ds in [ctrl, ctrl_A, ctrl_P, ctrl_I, ctrl_S, ctrl_M, ctrl_N]:
    for field in ['OHC_global', 'OHC_global_levels', 'OHC_zonal', 'OHC_zonal_levels']:
        ds[field][105] = (ds[field].sel({'time':204*365}) +
                                           ds[field].sel({'time':206*365}) )/2

rcp    = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Global_Ocean_rcp.nc'   , decode_times=False)
rcp_A  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Atlantic_Ocean_rcp.nc' , decode_times=False)
rcp_P  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Pacific_Ocean_rcp.nc'  , decode_times=False)
rcp_I  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Indian_Ocean_rcp.nc'   , decode_times=False)
rcp_M  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Mediterranean_rcp.nc'  , decode_times=False)
rcp_S  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Southern_Ocean_rcp.nc' , decode_times=False)
rcp_N  = xr.open_dataset(f'{path_samoc}/OHC/OHC_integrals_Arctic_Ocean_rcp.nc'   , decode_times=False)

In [ ]:
ctrl

In [ ]:
# long term, files calculated in OHC.ipynb
OHC_vert_trend_c1  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_c1.nc' , decode_times=False)
OHC_vert_trend_c2  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_c2.nc' , decode_times=False)
OHC_vert_trend_rcp = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_rcp.nc', decode_times=False)

OHC_vert_trend_c1  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_a_c1.nc' , decode_times=False)
OHC_vert_trend_c2  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_a_c2.nc' , decode_times=False)
OHC_vert_trend_rcp = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_a_rcp.nc', decode_times=False)

OHC_vert_trend_c1  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_b_c1.nc' , decode_times=False)
OHC_vert_trend_c2  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_b_c2.nc' , decode_times=False)
OHC_vert_trend_rcp = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_b_rcp.nc', decode_times=False)

In [ ]:
# decadal trend
OHC_vert_trend_rcp_decadal = ocn_field_regression(rcp .OHC_vertical[30:40,:,:]   )/spy

In [ ]:
xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_c2.nc', decode_times=False).plot()

In [ ]:
# combining picures
from PIL import Image
names = ['full_depth', 'above_100m', 'below_100m']
names2 = ['full\ndepth', 'above\n100m', 'below\n100m']
width  = 800
height = 500

In [ ]:
domain = 'ocn_T'
for run in ['ctrl', 'rcp']:
    if run=='ctrl':
        text1 = '<CTRL2>'
        OHC_vt   = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_c2.nc'    , decode_times=False)
        OHC_vt_a = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_a_c2.nc'  , decode_times=False)
        OHC_vt_b = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_b_c2.nc'  , decode_times=False)

    elif run=='rcp':
        text1 = '<RCP>'
        OHC_vt   = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_rcp.nc'   , decode_times=False)
        OHC_vt_a = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_a_rcp.nc' , decode_times=False)
        OHC_vt_b = xr.open_dataarray(f'{path_samoc}/OHC/OHC_trend_vert_int_b_rcp.nc' , decode_times=False)

    vert_trends = [OHC_vt, OHC_vt_a, OHC_vt_b]
    
    for i, name in enumerate(names):  # rows: full depth, top 100m, below 100m
        text2 = names2[i]
        fn = f'{path_results}/OHC/OHC-video/OHC_trend_vert_int_{name}_ref_map_{run}.png'
        xa = vert_trends[j]#.mean(dim='time')
        if i in [0,2]:  
            minv  = -3
            maxv  = 9
            nc    = 12
        elif i==1:  
            minv  = -2
            maxv  = 6
            nc    = 8

        label = r'linear OHC trend [W/m$^2$]'
        cmap  = discrete_cmap(nc, shifted_color_map(cmocean.cm.balance, 
                                                        start=.33, midpoint=0.5,
                                                        stop=1., name='shrunk'))
        f, ax = map_eq_earth(xa=xa, domain=domain, cmap=cmap, minv=minv, maxv=maxv,
                                 label=label, filename=fn, text1=text1, text2=text2)

In [ ]:
%%time

for run in ['ctrl', 'rcp']:
    if run=='ctrl':
        nt = 160
    elif run=='rcp':
        nt = 69
    for t in np.arange(1,nt+1):
        new_im = Image.new('RGB', (3*width, 3*height))
        print(t)
        for i, name in enumerate(names):  # rows: full depth, top 100m, below 100m
            for j in range(3):  # columns
                if j==0:    # reference field
                    fn = f'{path_results}/OHC/OHC-video/OHC_trend_vert_int_{name}_ref_map_{run}.png'
                elif j==1:  # field(t),
                    fn = f'{path_results}/OHC/OHC-video/OHC_trend_vert_int_{name}_map_{run}_{t}.png'
                elif j==2:  # anomaly(t)
                    fn = f'{path_results}/OHC/OHC-video/OHC_trend_vert_int_{name}_anom_map_{run}_{t}.png'
                assert os.path.exists(fn)
                im = Image.open(fn)
                new_im.paste(im, (j*width,i*height))

        new_im.save(f'{path_results}/OHC/OHC-video/OHC_trend_vert_int_maps_{run}_{t}.png', format='png', author='Andre Jueling')

In [ ]:
new_im

In [ ]:
Image.open(f'{path_results}/OHC/OHC-video/OHC_trend_vert_int_{name}_ref_map_{run}.png').size

In [ ]:
! module load ffmpeg

In [ ]:
! cp ../../../LEC/doc/video_command.txt ../../doc/

`module load ffmpeg`
`ffmpeg -framerate 12 -start_number 1 -i HC_trend_vert_int_maps_ctrl%d.png -r 12 -qscale:v 0 OHC_trend_vert_int_maps_ctrl.avi`

## running mean of $\Delta$OHC

In [ ]:
# # files are written out with ../run/run_OHC_vert_diff_mean.py script
# OHC_vert_diff_mean_ctrl = xr.open_dataarray(f'{path_samoc}/OHC/OHC_vert_diff_mean_ctrl.nc')
# OHC_vert_diff_mean_rcp  = xr.open_dataarray(f'{path_samoc}/OHC/OHC_vert_diff_mean_rcp.nc' )
# OHC_vert_diff_rm_ctrl   = xr.open_dataarray(f'{path_samoc}/OHC/OHC_vert_diff_rm_ctrl.nc'  )
# OHC_vert_diff_rm_rcp    = xr.open_dataarray(f'{path_samoc}/OHC/OHC_vert_diff_rm_rcp.nc'   )

In [ ]:
dOHC_vert_mean_ctrl.plot(cmap=cmocean.cm.balance)

In [ ]:
dOHC_vert_ctrl[1].plot(vmin=-5e9,vmax=5e9, cmap=cmocean.cm.balance)

In [ ]:
dOHC_vert_ctrl

In [ ]:
for i in range(800):
    fn =f'{path_results}/CICE/CICE_XMXL_video/CICE_XMXL_ctrl_no_{i:04d}.png'
    if os.path.exists(fn)==False:
        print(fn, os.path.exists(fn))
        plt.scatter(i,i)
#         os.rename(fn, f'{fn}.png')

In [ ]:
os.path.exists(f'{path_results}/CICE/CICE_XMXL_video/CICE_XMXL_rcp_no_0013.png')

In [ ]:
from CICE import 

In [ ]:
(ctrl.OHC_vertical.max(dim='time')-ctrl.OHC_vertical.min(dim='time')).plot()

In [ ]:
plt.figure(figsize=(12,8))
(rcp.OHC_vertical.max(dim='time')-rcp.OHC_vertical.min(dim='time')).plot()

In [ ]:
plt.figure(figsize=(12,8))
((rcp.OHC_vertical.max(dim='time')-rcp.OHC_vertical.min(dim='time'))-
 (ctrl.OHC_vertical.max(dim='time')-ctrl.OHC_vertical.min(dim='time'))).plot(vmin=-5e9,vmax=5e9,cmap=cmocean.cm.balance)

In [ ]:
plt.figure(figsize=(12,8))
((rcp.OHC_vertical.max(dim='time'))-
 (ctrl.OHC_vertical.max(dim='time'))).plot(vmin=-5e9,vmax=5e9,cmap=cmocean.cm.balance)

In [ ]:
plt.figure(figsize=(12,8))
(rcp.OHC_vertical.min(dim='time')-
 ctrl.OHC_vertical.min(dim='time')).plot(vmin=-5e9,vmax=5e9,cmap=cmocean.cm.balance)

In [ ]:
maxv=1.5e10
f, ax = plt.subplots(1,2,figsize=(12,5))
(ctrl.OHC_vertical.max(dim='time')-ctrl.OHC_vertical.mean(dim='time')).plot(ax=ax[0], vmin=0,vmax=maxv, cmap=cmocean.cm.amp)
(ctrl.OHC_vertical.min(dim='time')-ctrl.OHC_vertical.mean(dim='time')).plot(ax=ax[1], vmin=-maxv,vmax=0, cmap=cmocean.cm.tempo_r)

In [ ]:
maxv=1.2e10
f, ax = plt.subplots(1,2,figsize=(12,5))
(rcp.OHC_vertical.max(dim='time')-rcp.OHC_vertical.mean(dim='time')).plot(ax=ax[0], vmin=0,vmax=maxv, cmap=cmocean.cm.amp)
(rcp.OHC_vertical.min(dim='time')-rcp.OHC_vertical.mean(dim='time')).plot(ax=ax[1], vmin=-maxv,vmax=0, cmap=cmocean.cm.tempo_r)

In [ ]:
(rcp.time/365).plot()

In [ ]:
plt.figure(figsize=(12,4))
for y in np.arange(2000,2064):
    plt.scatter(y+2, rcp.OHC_global.sel({'time':slice(365*y,365*(y+5))}).mean(dim='time').item() )
plt.plot(rcp.time/365, rcp.OHC_global.rolling(time=5, center=True).mean().values )

In [ ]:
OHC_vert = (rcp.OHC_vertical.max(dim='time')-rcp.OHC_vertical.min(dim='time'))-(ctrl.OHC_vertical.max(dim='time')-ctrl.OHC_vertical.min(dim='time'))
OHC_vert = OHC_vert.where(ds.REGION_MASK>0)

In [ ]:
map_robinson(OHC_vert, 'ocn_U', cmocean.cm.balance, -maxv, maxv, 'OHC [J/m^2]')

In [ ]:
%%time
OHC_vert_trend = xr_linear_trends_2D(ctrl.OHC_vertical, ('nlat', 'nlon'))

In [ ]:
OHC_vert_trend.plot()